## تعبیه‌ها

در مثال قبلی، ما روی بردارهای کیسه‌ای از کلمات با ابعاد بالا و طول `vocab_size` کار کردیم و به طور صریح از بردارهای نمایشی موقعیتی با ابعاد پایین به نمایش پراکنده یک‌داغ تبدیل می‌کردیم. این نمایش یک‌داغ از نظر حافظه کارآمد نیست، علاوه بر این، هر کلمه به صورت مستقل از کلمات دیگر در نظر گرفته می‌شود، یعنی بردارهای کدگذاری‌شده یک‌داغ هیچ شباهت معنایی بین کلمات را بیان نمی‌کنند.

در این بخش، ما به بررسی مجموعه داده **News AG** ادامه خواهیم داد. برای شروع، بیایید داده‌ها را بارگذاری کنیم و برخی از تعاریف را از دفترچه قبلی دریافت کنیم.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)
print("Vocab size = ",vocab_size)

Loading dataset...


d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\train.csv: 29.5MB [00:01, 18.8MB/s]                            
d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\test.csv: 1.86MB [00:00, 11.2MB/s]                          


Building vocab...
Vocab size =  95812


## مفهوم تعبیه چیست؟

ایده‌ی **تعبیه** این است که کلمات را با بردارهای متراکم و کم‌بعدی نمایش دهیم که به نوعی معنای مفهومی یک کلمه را منعکس می‌کنند. بعداً درباره‌ی نحوه‌ی ساخت تعبیه‌های معنایی بحث خواهیم کرد، اما فعلاً فقط به تعبیه‌ها به عنوان روشی برای کاهش ابعاد بردار کلمه فکر کنید.

بنابراین، لایه‌ی تعبیه یک کلمه را به عنوان ورودی دریافت می‌کند و یک بردار خروجی با اندازه‌ی مشخص `embedding_size` تولید می‌کند. به نوعی، این لایه بسیار شبیه به لایه‌ی `Linear` است، اما به جای دریافت بردار یک‌داغ (one-hot encoded)، می‌تواند شماره‌ی کلمه را به عنوان ورودی دریافت کند.

با استفاده از لایه‌ی تعبیه به عنوان اولین لایه در شبکه‌ی خود، می‌توانیم از مدل کیسه‌ی کلمات (bag-of-words) به مدل **کیسه‌ی تعبیه‌ها** (embedding bag) تغییر دهیم، جایی که ابتدا هر کلمه در متن خود را به تعبیه‌ی مربوطه تبدیل می‌کنیم و سپس یک تابع تجمعی مانند `sum`، `average` یا `max` را روی تمام این تعبیه‌ها محاسبه می‌کنیم.

![تصویری که یک طبقه‌بند تعبیه برای پنج کلمه‌ی دنباله را نشان می‌دهد.](../../../../../lessons/5-NLP/14-Embeddings/images/embedding-classifier-example.png)

شبکه‌ی عصبی طبقه‌بند ما با لایه‌ی تعبیه شروع می‌شود، سپس لایه‌ی تجمع و در نهایت یک طبقه‌بند خطی در بالای آن قرار می‌گیرد:


In [2]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

### مدیریت اندازه متغیر دنباله‌ها

به دلیل این معماری، مینی‌بچ‌ها برای شبکه ما باید به شیوه خاصی ایجاد شوند. در واحد قبلی، زمانی که از روش کیسه کلمات (bag-of-words) استفاده می‌کردیم، تمام تنسورهای BoW در یک مینی‌بچ اندازه‌ای برابر با `vocab_size` داشتند، بدون توجه به طول واقعی دنباله متن. اما وقتی به استفاده از تعبیه‌های کلمات (word embeddings) می‌پردازیم، تعداد کلمات در هر نمونه متن متغیر خواهد بود و هنگام ترکیب این نمونه‌ها در مینی‌بچ‌ها، باید از تکنیک پر کردن (padding) استفاده کنیم.

این کار را می‌توان با استفاده از همان تکنیک ارائه تابع `collate_fn` به منبع داده انجام داد:


In [3]:
def padify(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

### آموزش طبقه‌بند تعبیه

حالا که بارگذار داده مناسب را تعریف کرده‌ایم، می‌توانیم مدل را با استفاده از تابع آموزش که در واحد قبلی تعریف کرده‌ایم، آموزش دهیم:


In [4]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=1, epoch_size=25000)

3200: acc=0.6415625
6400: acc=0.6865625
9600: acc=0.7103125
12800: acc=0.726953125
16000: acc=0.739375
19200: acc=0.75046875
22400: acc=0.7572321428571429


(0.889799795315499, 0.7623160588611644)

> **توجه**: ما در اینجا فقط برای ۲۵ هزار رکورد آموزش می‌دهیم (کمتر از یک دوره کامل) به دلیل صرفه‌جویی در زمان، اما شما می‌توانید آموزش را ادامه دهید، یک تابع برای آموزش در چندین دوره بنویسید و با پارامتر نرخ یادگیری آزمایش کنید تا دقت بالاتری به دست آورید. شما باید بتوانید به دقت حدود ۹۰٪ برسید.


### لایه EmbeddingBag و نمایش دنباله‌های با طول متغیر

در معماری قبلی، لازم بود تمام دنباله‌ها را به یک طول یکسان پر کنیم تا بتوانیم آن‌ها را در یک دسته کوچک قرار دهیم. این روش کارآمدترین راه برای نمایش دنباله‌های با طول متغیر نیست - یک روش دیگر استفاده از بردار **offset** است که جابجایی‌های تمام دنباله‌ها را که در یک بردار بزرگ ذخیره شده‌اند، نگه می‌دارد.

![تصویری که نمایش دنباله با استفاده از offset را نشان می‌دهد](../../../../../lessons/5-NLP/14-Embeddings/images/offset-sequence-representation.png)

> **Note**: در تصویر بالا، یک دنباله از کاراکترها نشان داده شده است، اما در مثال ما با دنباله‌های کلمات کار می‌کنیم. با این حال، اصل کلی نمایش دنباله‌ها با استفاده از بردار offset همچنان یکسان باقی می‌ماند.

برای کار با نمایش offset، از لایه [`EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html) استفاده می‌کنیم. این لایه مشابه `Embedding` است، اما بردار محتوا و بردار offset را به عنوان ورودی می‌گیرد و همچنین شامل یک لایه میانگین‌گیری است که می‌تواند `mean`، `sum` یا `max` باشد.

در اینجا شبکه‌ای اصلاح‌شده که از `EmbeddingBag` استفاده می‌کند آورده شده است:


In [5]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

برای آماده‌سازی مجموعه داده برای آموزش، باید یک تابع تبدیل ارائه دهیم که بردار جابجایی را آماده کند:


In [6]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)

توجه داشته باشید که برخلاف تمام مثال‌های قبلی، شبکه ما اکنون دو پارامتر را می‌پذیرد: بردار داده و بردار آفست، که اندازه‌های متفاوتی دارند. به همین ترتیب، بارگذار داده ما نیز به جای ۲ مقدار، ۳ مقدار را فراهم می‌کند: هر دو بردار متن و آفست به عنوان ویژگی‌ها ارائه می‌شوند. بنابراین، باید تابع آموزش خود را کمی تنظیم کنیم تا به این موضوع رسیدگی کند:


In [7]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.CrossEntropyLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,text,off in dataloader:
        optimizer.zero_grad()
        labels,text,off = labels.to(device), text.to(device), off.to(device)
        out = net(text, off)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count


train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6153125
6400: acc=0.6615625
9600: acc=0.6932291666666667
12800: acc=0.715078125
16000: acc=0.7270625
19200: acc=0.7382291666666667
22400: acc=0.7486160714285715


(22.771553103007037, 0.7551983365323096)

## تعبیه‌های معنایی: Word2Vec

در مثال قبلی، لایه تعبیه مدل یاد گرفت که کلمات را به نمایش برداری تبدیل کند، اما این نمایش معنای زیادی از نظر معنایی نداشت. خوب است که چنین نمایش برداری را یاد بگیریم که کلمات مشابه یا مترادف‌ها به بردارهایی تبدیل شوند که از نظر فاصله برداری (مثلاً فاصله اقلیدسی) به یکدیگر نزدیک باشند.

برای انجام این کار، باید مدل تعبیه خود را به روش خاصی بر روی مجموعه بزرگی از متن پیش‌پردازش کنیم. یکی از اولین روش‌ها برای آموزش تعبیه‌های معنایی [Word2Vec](https://en.wikipedia.org/wiki/Word2vec) نام دارد. این روش بر اساس دو معماری اصلی است که برای تولید نمایش توزیع‌شده کلمات استفاده می‌شوند:

- **کیسه کلمات پیوسته** (CBoW) — در این معماری، مدل را آموزش می‌دهیم تا یک کلمه را از متن اطراف پیش‌بینی کند. با توجه به ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$، هدف مدل پیش‌بینی $W_0$ از $(W_{-2},W_{-1},W_1,W_2)$ است.
- **اسکیپ‌گرام پیوسته** برخلاف CBoW عمل می‌کند. مدل از پنجره‌ای از کلمات زمینه اطراف برای پیش‌بینی کلمه فعلی استفاده می‌کند.

CBoW سریع‌تر است، در حالی که اسکیپ‌گرام کندتر است اما در نمایش کلمات نادر بهتر عمل می‌کند.

![تصویری که الگوریتم‌های CBoW و Skip-Gram را برای تبدیل کلمات به بردارها نشان می‌دهد.](../../../../../lessons/5-NLP/14-Embeddings/images/example-algorithms-for-converting-words-to-vectors.png)

برای آزمایش تعبیه Word2Vec که بر روی مجموعه داده Google News پیش‌پردازش شده است، می‌توانیم از کتابخانه **gensim** استفاده کنیم. در زیر کلماتی که بیشترین شباهت را به 'neural' دارند پیدا می‌کنیم:

> **توجه:** زمانی که برای اولین بار بردارهای کلمات را ایجاد می‌کنید، دانلود آن‌ها ممکن است کمی زمان‌بر باشد!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [9]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


ما همچنین می‌توانیم تعبیه‌های برداری را از کلمه محاسبه کنیم تا در آموزش مدل طبقه‌بندی استفاده شود (ما فقط ۲۰ مؤلفه اول بردار را برای وضوح نشان می‌دهیم):


In [10]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

نکته عالی درباره تعبیه‌های معنایی این است که می‌توانید رمزگذاری برداری را برای تغییر معنا دستکاری کنید. برای مثال، می‌توانیم بخواهیم کلمه‌ای پیدا کنیم که نمایش برداری آن تا حد ممکن به کلمات *پادشاه* و *زن* نزدیک باشد و از کلمه *مرد* دور باشد:


In [10]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

هر دو مدل CBoW و Skip-Grams "پیش‌بینی‌کننده" هستند، به این معنا که فقط زمینه‌های محلی را در نظر می‌گیرند. Word2Vec از زمینه‌های کلی استفاده نمی‌کند.

**FastText** بر پایه Word2Vec ساخته شده است و با یادگیری نمایش‌های برداری برای هر کلمه و n-gram‌های کاراکتری موجود در هر کلمه کار می‌کند. مقادیر این نمایش‌ها در هر مرحله آموزش به یک بردار میانگین‌گیری می‌شوند. اگرچه این روش محاسبات اضافی زیادی به پیش‌آموزش اضافه می‌کند، اما به تعبیه‌های کلمات اجازه می‌دهد اطلاعات زیر-کلمه‌ای را رمزگذاری کنند.

روش دیگری به نام **GloVe** از ایده ماتریس هم‌رخداد استفاده می‌کند و با استفاده از روش‌های عصبی ماتریس هم‌رخداد را به بردارهای کلمه‌ای غیرخطی و بیانگرتر تجزیه می‌کند.

می‌توانید با تغییر تعبیه‌ها به FastText و GloVe با مثال بازی کنید، زیرا gensim از چندین مدل مختلف تعبیه کلمات پشتیبانی می‌کند.


## استفاده از تعبیه‌های از پیش آموزش‌دیده در PyTorch

می‌توانیم مثال بالا را تغییر دهیم تا ماتریس در لایه تعبیه خود را با تعبیه‌های معنایی مانند Word2Vec از پیش پر کنیم. باید در نظر داشته باشیم که واژگان تعبیه‌های از پیش آموزش‌دیده و متن ما احتمالاً با یکدیگر مطابقت ندارند، بنابراین وزن‌های مربوط به کلمات گم‌شده را با مقادیر تصادفی مقداردهی اولیه خواهیم کرد:


In [11]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.get_itos()):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

Embedding size: 300
Populating matrix, this will take some time...Done, found 41080 words, 54732 words missing


حالا بیایید مدل خود را آموزش دهیم. توجه داشته باشید که زمان لازم برای آموزش مدل به طور قابل توجهی بیشتر از مثال قبلی است، به دلیل اندازه بزرگ‌تر لایه تعبیه و بنابراین تعداد پارامترهای بسیار بیشتر. همچنین، به همین دلیل، ممکن است نیاز داشته باشیم مدل خود را روی مثال‌های بیشتری آموزش دهیم اگر بخواهیم از بیش‌برازش جلوگیری کنیم.


In [12]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6359375
6400: acc=0.68109375
9600: acc=0.7067708333333333
12800: acc=0.723671875
16000: acc=0.73625
19200: acc=0.7463541666666667
22400: acc=0.7560714285714286


(214.1013875559821, 0.7626759436980166)

در مورد ما، افزایش چشمگیری در دقت مشاهده نمی‌کنیم که احتمالاً به دلیل تفاوت‌های زیاد در واژگان است.  
برای غلبه بر مشکل تفاوت واژگان، می‌توانیم از یکی از راه‌حل‌های زیر استفاده کنیم:  
* مدل word2vec را با واژگان خودمان دوباره آموزش دهیم  
* مجموعه داده خود را با واژگان مدل از پیش آموزش‌دیده word2vec بارگذاری کنیم. واژگانی که برای بارگذاری مجموعه داده استفاده می‌شود، می‌تواند در هنگام بارگذاری مشخص شود.  

روش دوم به نظر ساده‌تر می‌آید، به‌ویژه به این دلیل که چارچوب `torchtext` در PyTorch پشتیبانی داخلی برای embeddings دارد. به عنوان مثال، می‌توانیم واژگان مبتنی بر GloVe را به شکل زیر ایجاد کنیم:  


In [14]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)

100%|█████████▉| 399999/400000 [00:15<00:00, 25411.14it/s]


واژگان بارگذاری شده عملیات پایه زیر را دارد:
* دیکشنری `vocab.stoi` به ما اجازه می‌دهد که کلمه را به شاخص دیکشنری آن تبدیل کنیم
* `vocab.itos` برعکس عمل می‌کند - عدد را به کلمه تبدیل می‌کند
* `vocab.vectors` آرایه‌ای از بردارهای تعبیه است، بنابراین برای دریافت تعبیه یک کلمه `s` باید از `vocab.vectors[vocab.stoi[s]]` استفاده کنیم

در اینجا مثالی از دستکاری تعبیه‌ها برای نشان دادن معادله **kind-man+woman = queen** آورده شده است (من مجبور شدم ضریب را کمی تغییر دهم تا کار کند):


In [15]:
# get the vector corresponding to kind-man+woman
qvec = vocab.vectors[vocab.stoi['king']]-vocab.vectors[vocab.stoi['man']]+1.3*vocab.vectors[vocab.stoi['woman']]
# find the index of the closest embedding vector 
d = torch.sum((vocab.vectors-qvec)**2,dim=1)
min_idx = torch.argmin(d)
# find the corresponding word
vocab.itos[min_idx]

'queen'

برای آموزش طبقه‌بند با استفاده از این تعبیه‌ها، ابتدا باید مجموعه داده خود را با استفاده از واژگان GloVe کدگذاری کنیم:


In [16]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1],voc=vocab)) for t in b] # pass the instance of vocab to encode function!
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

همانطور که در بالا مشاهده کردیم، تمام جاسازی‌های برداری در ماتریس `vocab.vectors` ذخیره می‌شوند. این کار را فوق‌العاده آسان می‌کند تا آن وزن‌ها را با استفاده از کپی ساده به وزن‌های لایه جاسازی بارگذاری کنیم:


In [17]:
net = EmbedClassifier(len(vocab),len(vocab.vectors[0]),len(classes))
net.embedding.weight.data = vocab.vectors
net = net.to(device)

حالا بیایید مدل خود را آموزش دهیم و ببینیم آیا نتایج بهتری می‌گیریم:


In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6271875
6400: acc=0.68078125
9600: acc=0.7030208333333333
12800: acc=0.71984375
16000: acc=0.7346875
19200: acc=0.7455729166666667
22400: acc=0.7529464285714286


(35.53972978646833, 0.7575175943698017)

یکی از دلایلی که ما افزایش قابل توجهی در دقت مشاهده نمی‌کنیم این است که برخی از کلمات موجود در مجموعه داده ما در واژگان از پیش آموزش‌دیده GloVe وجود ندارند و بنابراین عملاً نادیده گرفته می‌شوند. برای غلبه بر این موضوع، می‌توانیم تعبیه‌های خود را بر روی مجموعه داده خود آموزش دهیم.


## تعبیه‌های زمینه‌ای

یکی از محدودیت‌های اصلی نمایش‌های تعبیه‌ای پیش‌آموزش‌شده سنتی مانند Word2Vec، مشکل رفع ابهام معنای کلمات است. در حالی که تعبیه‌های پیش‌آموزش‌شده می‌توانند بخشی از معنای کلمات در زمینه را درک کنند، هر معنای ممکن یک کلمه در همان تعبیه رمزگذاری می‌شود. این موضوع می‌تواند در مدل‌های پایین‌دستی مشکلاتی ایجاد کند، زیرا بسیاری از کلمات مانند کلمه «play» بسته به زمینه‌ای که در آن استفاده می‌شوند، معانی متفاوتی دارند.

برای مثال، کلمه «play» در این دو جمله معانی کاملاً متفاوتی دارد:
- من به یک **نمایش** در تئاتر رفتم.
- جان می‌خواهد با دوستانش **بازی** کند.

تعبیه‌های پیش‌آموزش‌شده بالا هر دو معنای کلمه «play» را در یک تعبیه نمایش می‌دهند. برای غلبه بر این محدودیت، باید تعبیه‌هایی بر اساس **مدل زبان** بسازیم که بر روی یک مجموعه بزرگ از متن آموزش داده شده و *می‌داند* چگونه کلمات می‌توانند در زمینه‌های مختلف کنار هم قرار گیرند. بحث درباره تعبیه‌های زمینه‌ای خارج از محدوده این آموزش است، اما زمانی که درباره مدل‌های زبان در واحد بعدی صحبت کنیم، به آن‌ها بازخواهیم گشت.



---

**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما تلاش می‌کنیم دقت را حفظ کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است شامل خطاها یا نادرستی‌ها باشند. سند اصلی به زبان اصلی آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حساس، توصیه می‌شود از ترجمه حرفه‌ای انسانی استفاده کنید. ما مسئولیتی در قبال سوءتفاهم‌ها یا تفسیرهای نادرست ناشی از استفاده از این ترجمه نداریم.
